In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
import pandas as pd
import fitz  #PyMuPDF

def pdf_to_text(file_path):
    with fitz.open(file_path) as pdf:
        text = ''
        for page in pdf:
            text += page.get_text()
    return text


In [2]:
import os
import pandas as pd

pdf_directory_vw = #Your local directory to folder of volkswagen reports
pdf_directory_psa = #Your local directory to folder of groupePSA reports
pdf_directory_vc = #Your local directory to folder of volvo cars reports

file_list_vw = os.listdir(pdf_directory_vw)
file_list_psa = os.listdir(pdf_directory_psa)
file_list_vc = os.listdir(pdf_directory_vc)

data_vw_excel = []
data_psa_excel = []
data_vc_excel = []

In [3]:
#Convert CSif os.path.exists('CSVFiles'):
if os.path.exists('CSVFiles'):    
    df_vw_excel = pd.read_csv('CSVFiles/df_vw_2.csv')
    df_psa_excel = pd.read_csv('CSVFiles/df_psa_2.csv')
    df_vc_excel = pd.read_csv('CSVFiles/df_vc_2.csv')
else:
    print("CSVFiles directory does not exist.")

df_vc_excel["text"] = df_vc_excel["text"].astype(str)
df_vw_excel["text"] = df_vw_excel["text"].astype(str)
df_psa_excel["text"] = df_psa_excel["text"].astype(str)

In [ ]:
import os
import nltk

# Set the NLTK data path to your desired directory
nltk_data_path = os.path.join(os.path.expanduser("~"), "nltk_data")
nltk.data.path.append(nltk_data_path)

nltk.download("stopwords")
nltk.download("wordnet")

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

lemmatizer = WordNetLemmatizer()

#Download required NLTK resources
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("omw-1.4")

#the word which is to be analyzed
target_word = "development"

def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    #Tokenize the text
    words = nltk.word_tokenize(text)

    #Preprocess only the target word
    lemmatizer = WordNetLemmatizer()
    processed_words = []
    for word in words:
        #Remove punctuation and special characters from the word
        clean_word = re.sub(r"[^\w\s]", "", word)
        clean_word = clean_word.lower()
        clean_word = lemmatizer.lemmatize(clean_word)
        
        #Perform lemmatization only on the target word
        if clean_word == target_word:
            processed_words.append(clean_word)
        else:
            processed_words.append(word)

    return " ".join(processed_words)


#Preprocess text in each dataframe
df_vc_excel["processed_text"] = df_vc_excel["text"].apply(preprocess_text)
df_vw_excel["processed_text"] = df_vw_excel["text"].apply(preprocess_text)
df_psa_excel["processed_text"] = df_psa_excel["text"].apply(preprocess_text)

In [6]:
# Filter dataframes by year
df_vc_2021 = df_vc_excel[df_vc_excel["year"] == 2021]
df_vw_2021 = df_vw_excel[df_vw_excel["year"] == 2021]
df_psa_2021 = df_psa_excel[df_psa_excel["year"] == 2021]


In [7]:
import re

def get_target_word_context(text, target_word, context_length=50):
    words = nltk.word_tokenize(text)
    contexts = []
    
    for i, word in enumerate(words):
        if word == target_word:
            start = max(0, i - context_length)
            end = min(i + context_length + 1, len(words))
            context = words[start:end]
            contexts.append(" ".join(context))
    
    return contexts

#Create new dataframes with target word context
def create_context_df(df, target_word, context_length=50):
    context_data = []
    for index, row in df.iterrows():
        contexts = get_target_word_context(row["processed_text"], target_word, context_length)
        for context in contexts:
            context_data.append({"index": index, "context": context})
    
    return pd.DataFrame(context_data)

#Generate context dataframes for each filtered dataframe
df_vc_context = create_context_df(df_vc_2021, target_word)
df_vw_context = create_context_df(df_vw_2021, target_word)
df_psa_context = create_context_df(df_psa_2021, target_word)


In [8]:
#Save dataframes to Excel files
df_vc_context.to_excel("df_vc_context_development_2021.xlsx", index=False)
df_vw_context.to_excel("df_vw_context_development_2021.xlsx", index=False)
df_psa_context.to_excel("df_psa_context_development_2021.xlsx", index=False)
